In [2]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
matcher = Matcher(nlp.vocab)

In [25]:
matched_sents = []

In [26]:
pattern = [{"LOWER":"facebook"},{"LEMMA":"be"},{"POS":"ADV","OP":"*"},{"POS":"ADJ"}]

In [27]:
def callback_method_fb(matcher, doc , i, matches):
    matched_id, start, end = matches[i]
    span = doc[start:end]
    sent = span.sent
    
    match_ents = [{
        'start':span.start_char - sent.start_char,
        'end':span.end_char - sent.start_char,
        'label':'MATCH'
    }]
    matched_sents.append({'text':sent.text, 'ents':match_ents})

In [28]:
matcher.add("fb",callback_method_fb, pattern)

In [29]:
doc = nlp("I'd say that Facebook is evil. - Facebook is pretty cool, right?")

In [30]:
matches = matcher(doc)

In [31]:
matches

[(8017838677478259815, 4, 7), (8017838677478259815, 9, 13)]

In [32]:
matched_sents

[{'text': "I'd say that Facebook is evil.",
  'ents': [{'start': 13, 'end': 29, 'label': 'MATCH'}]},
 {'text': 'Facebook is pretty cool, right?',
  'ents': [{'start': 0, 'end': 23, 'label': 'MATCH'}]}]

In [33]:
displacy.render(matched_sents, style='ent', manual=True)

### Phone Number Matching

In [49]:
pattern_num = [{"ORTH":"("},{"SHAPE":"ddd"},{"ORTH":")"},{"SHAPE":"dddd"},{"ORTH":"-", "OP":"?"},{"SHAPE":"dddd"}]

In [57]:
matcher = Matcher(nlp.vocab)
matcher.add('PhoneNumber',None, pattern_num)

In [58]:
doc = nlp("Call me at (123) 4560 1234")

In [59]:
print([t.text for t in doc])

['Call', 'me', 'at', '(', '123', ')', '4560', '1234']


In [60]:
matches = matcher(doc)
matches

[(7978097794922043545, 3, 8)]

In [61]:
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)

(123) 4560 1234


### Email Address Matching

In [63]:
pattern_em = [{"TEXT":{"REGEX":"[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+"}}]

In [65]:
matcher = Matcher(nlp.vocab)
matcher.add("Email",None, pattern_em)

In [71]:
doc = nlp("Contact me at johncarter99@gmail.com and kevinhart245@yahoo.com")

In [72]:
print([t.text for t in doc])

['Contact', 'me', 'at', 'johncarter99@gmail.com', 'and', 'kevinhart245@yahoo.com']


In [73]:
matches = matcher(doc)
matches

[(11010771136823990775, 3, 4), (11010771136823990775, 5, 6)]

In [74]:
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)

johncarter99@gmail.com
kevinhart245@yahoo.com


### Hashtag and Emoji Detection